In [11]:
import os
import re

In [12]:
import spacy
from spacy.matcher import Matcher
#import en_core_web_lg
import en_core_web_sm


In [14]:
!python -m spacy download en_core_web_sm


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [15]:
!pip install tika


In [16]:
from tika import parser 


In [17]:



def preprocess(pt):
  pt = re.sub('\[.*\]','',pt)
  pt = re.sub('\(.*\)','',pt)
  pt = re.sub('\n','',pt)
  pt = re.sub('Order\%.*\%.',"",pt)
  pt = re.sub('Microsoft Word',"",pt)  
  pt = re.sub('\sPage\s[0-9]\sof\s[0-9]?[0-9]\s','',pt)
  pt = re.sub('.*docx','',pt)
  pt = re.sub('_*_','',pt)
  pt = re.sub('/',' ',pt)
  pt = re.sub('\sPage\s[0-9]','',pt)
  pt = re.sub('[0-9]?[0-9]\.','',pt)
  pt = pt.lstrip()
  pt = pt.rstrip()
  return pt


In [18]:
def footer_preprocess(pt):
  pt = re.sub('\n',' ',pt)
  pt = re.sub('Order\%.*\%.',"",pt)
  pt = re.sub('\sPage\s[0-9]\sof\s[0-9]?[0-9]\s',"",pt)
  pt = pt.lstrip()
  pt = pt.rstrip()
  pt = " ".join(pt.split())
  return pt


In [19]:
all_files = []
is_found = False
#change dir to point to the folder with the orders 
for path,dirname,filenames in os.walk('/content/drive/My Drive/SEBI /Orders'):
  for names in filenames:
    all_files.append(os.path.join(path,names))

all_regulation_text = []
for document in all_files:
  is_found == False
  raw = parser.from_file(document)
  text = raw['content']
  #All AO orders start with a background subsection 
  header_index = text.lower().find('background')
  if header_index == -1:
    header_index = text.lower().find('facts')
  #print(header_index)
  text_header = text[:header_index]
  text = text[header_index:]
  points_split = re.split("\n[0-9]?[0-9]\.\s",text)
  num_of_points = len(points_split)
  #print(text_header)
  nlp = en_core_web_sm.load()
  matcher = Matcher(nlp.vocab)
  regulation_text_point = []
  for pt in points_split[1:num_of_points-1]:
    is_found = False 
    pt = re.sub('\n',' ',pt)
    doc = nlp(pt)
    tokens = [token.text for token in doc]
    pattern_regulations_one = [{'LOWER': 'a.'},
              {'LOWER': 'whether'}]
    pattern_regulations_two = [{'ORTH': 'a'},
           {'IS_PUNCT': True},
           {'IS_PUNCT': True, 'OP': '?'},
           {'LOWER': 'whether'}]
    pattern_regulations_three = [{'LOWER': '1.'},
              {'LOWER': 'whether'}]
    pattern_regulations_four = [{'ORTH': '1'},
           {'IS_PUNCT': True},
           {'LOWER': 'whether'}]
    pattern_regulations_five = [{'LOWER': '1)'},
              {'LOWER': 'whether'}]
    pattern_regulations_six = [{'ORTH': 'i'},
           {'IS_PUNCT': True},
           {'IS_PUNCT': True, 'OP': '?'},
           {'LOWER': 'whether'}]
    pattern_regulations_seven = [{'IS_PUNCT': False, 'IS_ALPHA': False, 'IS_DIGIT': False, 'IS_ASCII': False},
           {'LOWER': 'whether'}]
    matcher.add("RegRule1",None,pattern_regulations_one)
    matcher.add("RegRule2",None,pattern_regulations_two)
    matcher.add("RegRule3",None,pattern_regulations_three)
    matcher.add("RegRule4",None,pattern_regulations_four)
    matcher.add("RegRule5",None,pattern_regulations_five)
    matcher.add("RegRule6",None,pattern_regulations_six)
    matcher.add("RegRule7",None,pattern_regulations_seven)

    matches = matcher(doc)
    if len(matches)>0:
      #print("found")
      regulation_text_point.append(pt)
  all_regulation_text.append((regulation_text_point,document))



  
  

   

2020-08-22 09:38:32,371 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2020-08-22 09:38:33,199 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2020-08-22 09:38:33,704 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [20]:
for regtext in all_regulation_text:
  total_ents = []
  print(regtext[0])
  print(regtext[1],end="\n")


['On perusal of the Show Cause Notice, and other material available on record, I have the  following issues for consideration, viz,      (1) Whether the noticee has violated Regulations 3 (a) (b) (c) & (d), 4 (1), (2) (a) (d) (k) and   (r) of SEBI (PFUTP) Regulations 2003.  (2) Whether the noticee has violated Section 11C(3) read with 11C(2) of the SEBI Act.  (3) Whether the noticee has violated Regulation 13(4) read with 13(5) of SEBI (PIT)   Regulations, 1992  (4) Whether the noticee is liable for monetary penalty under sections. 15HA and 15A (a) &   (b) of SEBI Act, 1992. .  (5) What quantum of monetary penalty should be imposed on the noticee, taking into   consideration the factors mentioned in section 15J of SEBI Act     FINDINGS  ']
/content/drive/My Drive/SEBI /Orders/1404287891786.pdf
['I have carefully perused the documents available on record and   the submissions of the Noticee. The issues that arise for   consideration in the present case are:   a. Whether the Noticee has 

In [26]:
for document in all_files:
  raw = parser.from_file(document)
  text = raw['content']
  #All AO orders start with a background subsection 
  header_index = text.lower().find('background')
  header_index = text.lower().find('background')
  if header_index == -1:
    header_index = text.lower().find('facts')
  #print(header_index)
  text_header = text[:header_index]
  text = text[header_index:]
  points_split = re.split("\n[0-9]?[0-9]\.\s",text)
  num_of_points = len(points_split)
  #print(text_header)
  nlp = en_core_web_sm.load()
  matcher = Matcher(nlp.vocab)
  pre_processed_header = preprocess(text_header)
  doc = nlp(pre_processed_header)
  pattern_names = [{"LOWER": "in"}, {"LEMMA": "respect"},{"LEMMA": "of"}]
  matcher.add("Names", None, pattern_names)
  matches = matcher(doc)
  for match_id, start, end in matches:
    names_start = end 

  pattern_company = [{"LOWER": "in"}, {"LEMMA": "the"},{"LEMMA": "matter"}, {"LEMMA": "of"}]
  matcher.add("Names", None, pattern_company)
  matches = matcher(doc)
  for match_id, start, end in matches:
    names_end = start
    company_starts = end

  tokens = [token.text for token in doc] 
  print("Results using rule based approach")
  print(f'The names of the Noticees is/are {" ".join(tokens[names_start:names_end])}')
  print(f'The name of the company is {" ".join(tokens[company_starts:])}')
  print("Results using Spacy NER")
  names_spacy = [ent for ent in doc.ents if ent.label_ == "PERSON"]
  print(f'Names extraced from the header: {names_spacy}')
  Organization_spacy = [ent for ent in doc.ents if ent.label_ == "ORG"]
  print(f'Names of orgs in the header {Organization_spacy}')

  footer_pre_processed = footer_preprocess(points_split[num_of_points-1])
  #print(footer_pre_processed)

  doc = nlp(footer_pre_processed)
  pattern_place = [{"LOWER": "place"}, {'IS_PUNCT': True}]
  matcher.add("place", None, pattern_place)
  matches = matcher(doc)
  if len(matches)>0:
    for match_id, start, end in matches:
      place_index = end
    tokens = [token.text for token in doc]
    print("Using rule based approach")
    print(f"The place is {tokens[place_index]}")
  else: 
    pass
  #Only complete rule 
  pattern_date = [{'LOWER': 'date'},
            {'IS_PUNCT': True},
            {'ENT_TYPE': 'DATE'},
            {'ENT_TYPE': 'DATE'},
            {'IS_PUNCT': True},
            {'IS_DIGIT': True}]
  matcher.add("date",None,pattern_date)
  matches = matcher(doc)
  for match_id, start, end in matches:
    if tokens[start+2:end]:
      print(f"The date is {' '.join(tokens[start+2:end])}",end='\n\n')
  print(f'Doc name: {document.split("/")[-1]}')
  print('***************************************************************************')

  


  

  


  


Results using rule based approach
The names of the Noticees is/are   Mahesh Thanvi   
The name of the company is E.Star Infotech Ltd.    BRIEF
Results using Spacy NER
Names extraced from the header: [Mahesh Thanvi]
Names of orgs in the header [Infotech Ltd., SECURITIES AND EXCHANGE BOARD OF INDIA, SECURITIES AND EXCHANGE BOARD OF INDIA ACT, PROCEDURE FOR HOLDING]
Using rule based approach
The place is Mumbai
The date is March 05 , 2012

Doc name: 1404287891786.pdf
***************************************************************************
Results using rule based approach
The names of the Noticees is/are   Mr. Senthil Kumar U.P.                                
The name of the company is             M s Orchid Chemicals and Pharmaceuticals Limited
Results using Spacy NER
Names extraced from the header: [Senthil Kumar, Senthil Kumar]
Names of orgs in the header [Orchid Chemicals and, SECURITIES AND EXCHANGE BOARD OF INDIA ACT, SECURITIES AND EXCHANGE BOARD OF INDIA, PROCEDURE FOR HOLDING